In [104]:
import numpy as np
import pandas as pd
import warnings
import math

In [146]:
def to1D(listOf3DArray, listOfName, listOfCond):
    
    if not (len(listOf3DArray) == len(listOfName) == len(listOfCond)):
        raise Exception("lists of array, name, condition have different dimensions")
    
    output = pd.DataFrame()
    
    arr0 = listOf3DArray[0]
    x = arr0.shape[0]; y = arr0.shape[1]; z = arr0.shape[2]
    dimension = (x,y,z)
    
    for i in range(len(listOfName)):
        name = listOfName[i]
        arr = listOf3DArray[i]
        cond = listOfCond[i]
        
        # check the dimension of the input array
        if arr.shape[0] != x:
            raise Exception(str(i+1)+"th array has wrong x dimension")
        if arr.shape[1] != y:
            raise Exception(str(i+1)+"th array has wrong y dimension")
        if arr.shape[2] != z:
            raise Exception(str(i+1)+"th array has wrong z dimension")            
        # check the name of the input name
        if name not in ["CBF","OEF","R2p","BOLD","CMRO2","lobe", "nw", "CBV"]:
            warnings.warn(name + " is not in the default name list \n"+
                          "Default names are : CBF, OEF, R2p, BOLD, CMRO2, lobe, nw, CBV")
        
        output[name+cond] = arr.flatten()
    
    output.dropna(inplace = True)
    
    numEntity = x*y*z
    indexMap = np.arange(numEntity).reshape(x,y,z)
    
    return output, dimension, indexMap

In [147]:
def AverageFileReadandFilter(fileName, cond, IDs = None, thres = 0, rename = True):
    df = pd.read_csv(fileName, index_col=0)
    if IDs:
        df = df.loc[df['ID'].isin(IDs)]
    if thres:
        df = df.loc[df['vox_count'] >= thres]
    df = df.loc[(df['cond'].isin(cond)) | (df['cond'] == cond[0]+cond[1])]
    df["par_cond"] = df["par"]+df["cond"]
    df = df.pivot_table(index=["ID","parcel","nw","lobe"], columns=["par_cond"],values="median")
    df.dropna(inplace = True, subset=['BOLD'+cond[0]+cond[1]])
    if rename:
        df.rename({'xCBF_3_BRmsk_CSF'+cond[0]: 'CBF'+cond[0],
                    'xCBF_3_BRmsk_CSF'+cond[1]: 'CBF'+cond[1],
                    'sR2strich'+cond[0]: 'R2p'+cond[0],
                    'sR2strich'+cond[1]: 'R2p'+cond[1],
                    'rOEF'+cond[0]: 'OEF'+cond[0],
                    'rOEF'+cond[1]: 'OEF'+cond[1]}, axis=1, inplace=True)
    df.reset_index(inplace=True)
    return df

In [106]:
def to3D(df, listOfColumnName, dimension):
    DictOfOutput = {}
    for col in listOfColumnName:
        DictOfOutput[col] = np.array(df[col]).reshape(dimension)
    return DictOfOutput

In [107]:
def getRange(listOf3DCoord, indexMap):
    listOf1DCoord = []
    for coord in listOf3DCoord:
        listOf1DCoord.append(indexMap[coord[0]][coord[1]][coord[2]])
    return listOf1DCoord

In [108]:
def getRegion(df, listOfRegion, regionType):
    return list(df.loc[df[regionType].isin(listOfRegion)].index)

In [109]:
def getColName(func):
    colNameMap = {M:("M",[1]),
                 N:("N",[0,1]),
                 A:("a",[0]),
                 FickCMRO2:("FCMRO2",[0]),
                 FickCMRO2Ctrl:("FCMRO2",[1]),
                 FickCMRO2RelChange: ("rcFCMRO2",[0,1]),
                 DavisCMRO2RelChange: ("rcDCMRO2",[0,1]),
                 DavisCBFRelChange: ("rcDCBF",[0,1]),
                 DavisBOLD: ("DBOLD",[0,1]),
                 FujitaR2pRelChange: ("rcFR2P",[0,1]),
                 FujitaCMRO2RelChange: ("rcFCMRO2",[0,1])}
    return colNameMap[func]

In [110]:
def preparation(cond, TE):
    global condG; global TEG; global aG; global bG; global MG
    condG = cond
    TEG = TE
    aG = bG = MG = None

In [111]:
def applyFuncToDf(func, df, applyRange = [], 
                  TE = None, a = None, b = None, m = None,
                 colName = None):
    if not colName:
        funName, condList = getColName(func)
        colName = funName
        for i in condList:
            colName += condG[i]
    global TEG; global aG; global bG; global MG
    if TE:
        TEG = TE
        print("TE is changed")
    aG = a; bG = b; MG = m
    if a:
        colName += "_a"+str(a)
    if b:
        colName += "_b"+str(b)
    if m:
        colName += "_M"+str(M)
    if not applyRange:
        df[colName] = df.apply(func,axis=1)
    else:
        df.at[applyRange, colName] = (df.loc[set(applyRange)]).apply(func,axis=1)
    aG = bG = MG = None

In [141]:
def M(row):
    if not TEG:
        raise Exception("Trying to calculate M but TE is not specified, please add argument TE = ?")
    return row["R2p"+condG[1]]*TEG/10

In [113]:
def N(row):
    dCBF = row["CBF"+condG[0]] - row["CBF"+condG[1]]
    dCMRO2 = row["CMRO2"+condG[0]] - row["CMRO2"+condG[1]]
    return dCBF/dCMRO2

In [114]:
def A(row):
    return math.log(row["CBV"+condG[0]]/0.8)/math.log(row["CBF"+condG[0]])

In [115]:
def FickCMRO2(row):
    return row["CaO2"+condG[0]]*row["OEF"+condG[0]]*row["CBF"+condG[0]]

In [116]:
def FickCMRO2Ctrl(row):
    return row["CaO2"+condG[1]]*row["OEF"+condG[1]]*row["CBF"+condG[1]]

In [117]:
def FickCMRO2RelChange(row):
    return row["OEF"+condG[0]]*row["CBF"+condG[0]]/(row["OEF"+condG[1]]*row["CBF"+condG[1]])-1

In [145]:
def DavisCheck(row):
    a = aG; b = bG; m = MG
    if (not a) and ("a"+condG[0] not in row.index):
        warnings.warn(str("alpha is not specified and a"+condG[0] + " is not in the dataframe\n"+
                      "Calculating alpha based on CBF and CBV now"))
        a = A(row)
    elif not a:
        a = row["a"+condG[0]]
    if (not b):
        raise Exception("beta is not specified")
    if (not m) and ("M"+condG[1] not in row.index):
        warnings.warn(str("M is not specified and M"+condG[1] + " is not in the dataframe\n"+
                      "Calculating M based on R2p and TE now"))
        m = M(row)
    elif not m:
        m = row["M"+condG[1]]
    return a, b, m

In [120]:
def DavisCMRO2RelChange(row):
    a, b, m = DavisCheck(row)
    product = 1 - row["BOLD"+condG[0]+condG[1]]/m
    rCMRO2 = (product/((row["CBF"+condG[0]]/row["CBF"+condG[1]])**(a-b)))**(1/b)
    return rCMRO2-1    

In [121]:
def DavisBOLD(row):
    a, b, m = DavisCheck(row)
    return m*(1-(row["CBF"+condG[0]]/row["CBF"+condG[1]])**(a-b)*(row["CMRO2"+condG[0]]/row["CMRO2"+condG[1]])**b)-1

In [122]:
def DavisCBFRelChange(row):
    a, b, m = DavisCheck(row)
    product = 1 - row["BOLD"+condG[0]+condG[1]]/m
    rCBF = (product/((row["CMRO2"+condG[0]]/row["CMRO2"+condG[1]])**(b)))**(1/(a-b))
    return rCBF-1    

In [148]:
def FujitaR2pRelChange(row):
    return -row["BOLD"+condG[0]+condG[1]]/TEG/row["R2p"+condG[1]]

In [149]:
def FujitaCMRO2RelChange(row):
    ratioCBF = row["CBF"+condG[0]]/row["CBF"+condG[1]]
    ratioR2p = FujitaR2pRelChange(row)+1
    ratioCBV = row["CBV"+condG[0]]/row["CBV"+condG[1]]
    return ratioCBF*ratioR2p*ratioCBV-1

In [ ]:
def rc(var):
    return lambda row: (row[var+condG[0]] - row[var+condG[1]])/row[var+condG[1]]